In [ ]:
import warnings

import numpy as np
import optuna
import pandas as pd
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM
from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import MAP, Precision, Recall, calc_metrics
from rectools.models import (
    ImplicitALSWrapperModel,
    LightFMWrapperModel,
    PopularModel,
    RandomModel,
)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:
from enum import Enum


class ItemsFeatureTopKConfig(int, Enum):
    """Конфигурация для ограничения количества топовых значений фич."""

    DIRECTORS_TOP_K = 30
    STUDIOS_TOP_K = 15


In [ ]:
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [ ]:
interactions = pd.read_csv('interactions_processed.csv')
users = pd.read_csv('users_processed.csv')
items = pd.read_csv('items_processed.csv')

# Обработка данных

In [ ]:
Columns.Datetime = 'last_watch_dt'

In [ ]:
interactions.drop(
    interactions[interactions[Columns.Datetime].str.len() != 10].index,
    inplace=True,
)
interactions[Columns.Datetime] = pd.to_datetime(
    interactions[Columns.Datetime], format='%Y-%m-%d'
)
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 20, 3, 1)

In [ ]:
# Разделяем на train и test
train = interactions[
    interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)
].copy()
test = interactions[
    interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)
].copy()

In [ ]:
train.drop(train.query('total_dur < 300').index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])
len(cold_users)

In [ ]:
# Отбрасываем холодных пользователей
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Подготовка фич

## User features

In [ ]:
users

In [ ]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [ ]:
user_features_frames = []
for feature in ['sex', 'age', 'income']:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ['id', 'value']
    feature_frame['feature'] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

## Item features

In [ ]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [ ]:
items['genre'] = (
    items['genres'].str.replace(', ', ',', regex=False).str.split(',')
)
genre_feature = items[[Columns.Item, 'genre']].explode('genre')
genre_feature.columns = ['id', 'value']
genre_feature['feature'] = 'genre'
genre_feature.head()

In [ ]:
content_feature = items.reindex(columns=[Columns.Item, 'content_type'])
content_feature.columns = ['id', 'value']
content_feature['feature'] = 'content_type'

In [ ]:
countries_feature = items.reindex(columns=[Columns.Item, 'countries'])
countries_feature.columns = ['id', 'value']
countries_feature['feature'] = 'countries'

In [ ]:
release_decade_feature = items.reindex(columns=[Columns.Item, 'release_decade'])
release_decade_feature.columns = ['id', 'value']
release_decade_feature['feature'] = 'release_decade'

In [ ]:
release_decade_feature['value'].value_counts()

In [ ]:
age_rating_feature = items.reindex(columns=[Columns.Item, 'age_rating'])
age_rating_feature.columns = ['id', 'value']
age_rating_feature['feature'] = 'age_rating'

Берем только ТОП-K студий, а остальные заменяем на 'other_studio'. 'other_studio' и 'unknown_studio' - **разные** вещи!

In [ ]:
def replace_rare_studios(studio_list):
    return [
        studio if studio in top_studios else 'other_studio'
        for studio in studio_list
    ]


items['studio'] = items['studios'].str.split(r',\s*')
top_studios = (
    items['studio']
    .explode()
    .value_counts()
    .head(ItemsFeatureTopKConfig.STUDIOS_TOP_K)
    .index
)
items['studio'] = items['studio'].apply(replace_rare_studios)

In [ ]:
studios_feature = items[[Columns.Item, 'studio']].explode('studio')
studios_feature.columns = ['id', 'value']
studios_feature['feature'] = 'studios'

In [ ]:
# Для директоров оставляем топ-30, остальные заменяем на 'other'
items['directors'] = (
    items['directors'].str.replace(', ', ',', regex=False).str.split(',')
)
top_directors = (
    items['directors']
    .explode()
    .value_counts()
    .head(ItemsFeatureTopKConfig.DIRECTORS_TOP_K)
    .index
)

items['director'] = items['directors'].apply(
    lambda x: [d if d in top_directors else 'other_director' for d in x]
)

In [ ]:
directors_feature = items[[Columns.Item, 'director']].explode('director')
directors_feature.columns = ['id', 'value']
directors_feature['feature'] = 'director'
directors_feature.head()

In [ ]:
item_features = pd.concat((
    genre_feature,
    content_feature,
    countries_feature,
    release_decade_feature,
    age_rating_feature,
    studios_feature,
    directors_feature,
))


In [ ]:
item_features.info()

In [ ]:
CAT_USER_FEATURES = list(user_features['feature'].unique())
CAT_ITEM_FEATURES = list(item_features['feature'].unique())

In [ ]:
CAT_ITEM_FEATURES

In [ ]:
metrics_name = {
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [ ]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=CAT_USER_FEATURES,
    item_features_df=item_features,
    cat_item_features=CAT_ITEM_FEATURES,
)

TEST_USERS = test[Columns.User].unique()

# Подбор гиперпараметров

### Подбор гиперпараметров ImplicitALS

In [ ]:
def ials_objective(trial):
    factors = trial.suggest_categorical('factors', [4, 8, 16, 32, 64])
    regularization = trial.suggest_float('regularization', 0.001, 0.1, log=True)
    iterations = trial.suggest_categorical('iterations', [1, 3, 5, 10, 15])

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations=iterations,
            random_state=42,
            num_threads=4,
            use_gpu=False,
        ),
        fit_features_together=True,
    )

    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = calc_metrics(metrics, recos, test, train)['MAP@10']
    return map10

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(ials_objective, n_trials=20)

In [ ]:
print(f'Best MAP@10 value: {study.best_value}')
print(f'Best parameters: {study.best_params}')

In [ ]:
# Best MAP@10 value: 0.075254069127234
# Best parameters: {'factors': 32, 'regularization': 0.01883534498756549, 'iterations': 5}

### Подбор гиперпараметров LightFM

In [ ]:
def lfm_objective(trial):
    no_components = trial.suggest_categorical(
        'no_components', [8, 16, 32, 64, 128]
    )
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05, log=True)
    rho = trial.suggest_float('rho', 0.9, 0.99, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-6, 1e-5, log=True)

    model = LightFMWrapperModel(
        LightFM(
            no_components=no_components,
            learning_rate=learning_rate,
            rho=rho,
            epsilon=epsilon,
            user_alpha=0,
            item_alpha=0,
            random_state=42,
        ),
        epochs=1,
        num_threads=16,
    )
    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = calc_metrics(metrics, recos, test, train)['MAP@10']
    return map10

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(lfm_objective, n_trials=30)

# Лучшая модель

In [ ]:
model = ImplicitALSWrapperModel(
    model=AlternatingLeastSquares(
        factors=32,
        regularization=0.01883534498756549,
        iterations=5,
        random_state=42,
        num_threads=4,
        use_gpu=False,
    ),
    fit_features_together=True,
)

# Искусственные пользователи

Егор - любитель фильмов Квентина Тарантино

In [ ]:
users = pd.concat(
    [
        users,
        pd.DataFrame([
            {
                'user_id': 1100000,
                'age': 'age_18_24',
                'income': 'income_60_90',
                'sex': 'М',
                'kids_flg': 0,
            }
        ]),
    ],
    ignore_index=True,
)


In [ ]:
items.loc[
    items['item_id'].isin([14804, 7693, 11115, 8148, 16382, 4072, 898, 13715]),
    ['item_id', 'title', *CAT_ITEM_FEATURES],
]

In [ ]:
first_items = [14804, 7693, 11115, 8148, 16382, 4072, 898]
first_num_items = len(first_items)
first_avatar = pd.DataFrame({
    'user_id': np.full(first_num_items, fill_value=1100000),
    'item_id': first_items,
    'last_watch_dt': np.full(first_num_items, fill_value='2021-05-29'),
    'total_dur': np.full(first_num_items, fill_value=np.nan),
    'watched_pct': first_num_items * [100.0],
    'weight': [3, 3, 3, 3, 3, 3, 3],
})

first_avatar[Columns.Datetime] = pd.to_datetime(
    first_avatar[Columns.Datetime], format='%Y-%m-%d'
)

In [ ]:
avatars = pd.concat([first_avatar])
interactions = pd.concat([interactions, avatars], ignore_index=True)

In [ ]:
# Соберем датасет
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=CAT_USER_FEATURES,
    item_features_df=item_features,
    cat_item_features=CAT_ITEM_FEATURES,
)

avatars_ids = avatars['user_id'].unique()

In [ ]:
%%time
model.fit(dataset)

In [ ]:
recos = model.recommend(
    users=avatars_ids,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [ ]:
recs = recos.merge(
    items[['item_id', 'title', *CAT_ITEM_FEATURES]],
    on='item_id',
).sort_values(['user_id', 'rank'])

In [ ]:
recs[recs['user_id'] == avatars_ids[0]]